#  Veri Hazırlama

# 1. Veriyi alma

In [1]:
import pandas as pd

In [15]:
data = pd.read_csv('datasets/itchy_boots_raw.csv');

In [16]:
data.head()

,Unnamed: 0,season,episode,date,time,rating
0,0,SEASON 7 - Return to Africa,ITCHY BOOTS New Season: this is where I am go...,4 months,20,"s 1,237,258 views"
1,1,SEASON 7 - Return to Africa,I need their help. |S7 - E2|,4 months,19,"s 1,548,717 views"
2,2,SEASON 7 - Return to Africa,No motorcyclists have gone here before |S7 - E3|,4 months,25,"s 905,116 views"
3,3,SEASON 7 - Return to Africa,How I ended up in a Berber village in Morocco ...,4 months,18,"s 790,900 views"
4,4,SEASON 7 - Return to Africa,Motorcycling solo into the Sahara Desert of Mo...,4 months,18,"s 648,198 views"


# 2. Veriyi gözden geçirme ve kararlar

* Unnamed:0 gereksiz bir sütun
* season sütunundan sezon sayısı ve sezon ismini ayırmak gerekecek
* episode sütunundan da bölüm sayısı ve başlığı aıyrmalıyız
* date sütundaki year, month gibi ifadeleri silip, sadece yıl değerini almak, ay, hafta ve gün olan değerleri de bu yıla ait olacak şekilde 0 ile değiştirmek kararındayım
* time sütunu sorunzsuz gibi duruyor
* rating sütununda bazı satırlar "minutes" ifadesinden kalan 's' içeriyor bunları ve sondaki views ifadelerini silip sayısal dönüşüm yapılmalı

## 2.1. Unnamed:0

In [17]:
# Unnamed:0 index değerlerini özellik olarak almış, gerek yok. Basitçe onun dışarda bıraktım.
data= data.iloc[:, 1:]

In [18]:
data.head()

,season,episode,date,time,rating
0,SEASON 7 - Return to Africa,ITCHY BOOTS New Season: this is where I am go...,4 months,20,"s 1,237,258 views"
1,SEASON 7 - Return to Africa,I need their help. |S7 - E2|,4 months,19,"s 1,548,717 views"
2,SEASON 7 - Return to Africa,No motorcyclists have gone here before |S7 - E3|,4 months,25,"s 905,116 views"
3,SEASON 7 - Return to Africa,How I ended up in a Berber village in Morocco ...,4 months,18,"s 790,900 views"
4,SEASON 7 - Return to Africa,Motorcycling solo into the Sahara Desert of Mo...,4 months,18,"s 648,198 views"


## 2.2. Sezon isimleri ve numarası

"SEASON 7 - Ret...." şekklinde bir ifade biçimi var.  Önce '- ' ile isimle sayı bölümünü  ayırıp, sonra sayı bölümünü de boşluk karakteri ile ayırırsam sorun çözülüyor.

In [19]:
season_number = []
season_name = []
for s in data['season'].values:
    try:
        s1 = s.split('- ')
        season_number.append(int(s1[0].split(' ')[1]))
        season_name.append(s1[1])
    except:
        season_number.append(0)
        season_name.append(s1)

In [21]:
data['season_no'] = season_number
data['season_title'] = season_name

## 2.3. date

Bu özelliğe yaklaşımım: eğer metyin "year" ya da "years" içeriyorsa bu ifadeleri silerek sadece sayıyı alma şeklimnde.
Eğer bu iki ifade yoksa (month, week, day gibi ifadeler varsa) hiç bir şey yapmasına gerek yok, zaten 0.

In [26]:
import numpy as np

In [24]:
size = data.shape[0]
size

484

In [28]:
uploading_date = np.zeros(size)

In [30]:
for i in range(size):
    if 'years' in data.date[i]:
        uploading_date[i] = data.date[i].replace('years','').strip()
    elif 'year' in data.date[i]:
        uploading_date[i] = data.date[i].replace('year','').strip()
    

In [31]:
data['uploading_date'] = uploading_date.astype(int)

## 2.4. rating

Metin içindeki "views", bir önceki minutes'den kalan "s" ve sayıları ayıran "," ifadesini sildikten sonra bazı metinlerde bir sayı ve second ifadesinden kalan "econd" ("s" yi silmiştik) ile metni ikiye ayırırsak bunu da halletmiş olacağız.

In [33]:
ratings = data.rating

In [35]:
def create_rating(data):
    data_temp = [r.replace('views','') for r in data]
    data_temp1 = [r.replace(',','') for r in data_temp]
    data_temp2 = [r.replace('s','') for r in data_temp1]
    rate_list = []
    for r in data_temp2:
        if 'econd' in r:
            r = r.split('econd')[1]
        r = ''.join(r)
        r = r.strip()
        rate_list.append(int(r))
    return rate_list

In [36]:
rate_list = create_rating(ratings)

In [39]:
data['rate'] = rate_list

## 2.5. Bölüm sayısı ve başlığı

Bu bölümde biraz uğraşmak gerekiyor, farklı bölümlerde farklı patternler var. İki tane de sorunlu veri var.  Ayırıcı işaretler değişik, sıraları da farklı (bölüm sayıları başta ya da sonda). O yüzden sezon sezon ilerledim. Bir sezonda da ('Adventure Intermezzo: Tour of The Netherlands' bölüm sayısı yok) bölümleri sırasıyla numaraladım.

### 2.5.1. Her bölüm için veri ayırma

In [42]:
def split_data(data, first_split = ']', number_side = 0, second_split='Eps.', title_side = 1):
    """
    first_split: title ile S1E1 patternini ayıran bölme-> '[' karakteri ,']' karakteri  veya '|' olabilir.
    second_split: sayısal bölüm için -> 'Eps.', 'Eps', 'E' veya '-' olabilir.
    number_side: sayısal kısım başta kalmış ise 0 değilse 1
    title_side: başlık başta kalmış ise 0 değilse 1
    """
    
        
    if second_split not in data:
        second_split='Eps'
        if second_split not in data:
            second_split='E'
            if second_split not in data:
                second_split='-'
    if len(data.split(first_split)) > 1:
        if first_split =='[':
            t = data.split(first_split)[number_side].split(second_split)[1].strip()
            return int(t.replace(']','')), data.split(first_split)[title_side]
        else:
            return int(data.split(first_split)[number_side].split(second_split)[1].strip()), data.split(first_split)[title_side]
    else:
        return -1, data

In [44]:
# Bazı verileri elle düzelteecğim, uyarıları görmemek için
import warnings
warnings.filterwarnings('ignore')

In [45]:
s1 = data[data.season_no==1]['episode']

s1_episode_no = []
s1_episode_title = []

for i in s1:
    n,t = split_data(i)
    s1_episode_no.append(n)
    s1_episode_title.append(t)

In [46]:
s2 = data[data.season_no==2]['episode']

s2_episode_no = []
s2_episode_title = []

for i in s2:
    n,t = split_data(i)
    s2_episode_no.append(n)
    s2_episode_title.append(t)

In [47]:
s3 = data[data.season_no==3]['episode']

s3_episode_no = []
s3_episode_title = []

for i in s3:
    n,t = split_data(i, first_split='[', number_side=1, title_side=0)
    s3_episode_no.append(n)
    s3_episode_title.append(t)

In [48]:
s4 = data[data.season_no==4]['episode']

s4_episode_no = []
s4_episode_title = []

for i in s4:
    n,t = split_data(i, first_split='[', number_side=1, title_side=0)
    s4_episode_no.append(n)
    s4_episode_title.append(t)

In [49]:
s5 = data[data.season_no==5]['episode']

s5_episode_no = []
s5_episode_title = []

for i in s5:
    # print(i)
    n,t = split_data(i, first_split='[', number_side=1, title_side=0)
    s5_episode_no.append(n)
    s5_episode_title.append(t)

In [50]:
s6 = data[data.season_no==6]['episode']

In [55]:
# Sıkıntılı bir veriydi, elle  düzelttim
s6.iloc[66] = 'Only my motorcycle can pass here:I have to find an IXIL house in GUATEMALA |S6-E67| '

In [52]:
s6_episode_no = []
s6_episode_title = []

for i in s6:
    n,t = split_data(i, first_split='|', number_side=1, title_side=0)
    s6_episode_no.append(n)
    s6_episode_title.append(t)

In [54]:
s7 = data[data.season_no==7]['episode']

In [57]:
# İkinci sıkıntılı veri
s7.iloc[26] = 'First leg of the African Continent COMPLETE - Reaching DAKAR |S7 - Eps.27| '

In [58]:
s7_episode_no = []
s7_episode_title = []

for i in s7:
    n,t = split_data(i, first_split='|', number_side=1, title_side=0)
    s7_episode_no.append(n)
    s7_episode_title.append(t)

In [59]:
# Adventure intermezzo için
s_inter = data[data.season=='Adventure Intermezzo: Tour of The Netherlands']['episode']
s_inter_episode_title  = s_inter.values
s_inter_episode_no = []

for i in range(len(s_inter_episode_title)):
    s_inter_episode_no.append(i+1)

### 2.5.2. index değerleri

In [60]:
s1_index = data[data.season_no==1].index
s2_index = data[data.season_no==2].index
s3_index = data[data.season_no==3].index
s4_index = data[data.season_no==4].index
s5_index = data[data.season_no==5].index
s6_index = data[data.season_no==6].index
s7_index = data[data.season_no==7].index
s_inter_index = data[data.season=='Adventure Intermezzo: Tour of The Netherlands'].index

### 2.5.3. DataFrame çevrimi

In [61]:
def to_df(episodes, titles, index_):
    df = pd.DataFrame(episodes, index=index_)
    df['name']= titles
    df.columns = ['E','title']
    return df

In [62]:
df_season1 = to_df(s1_episode_no,s1_episode_title, s1_index)
df_season2 = to_df(s2_episode_no,s2_episode_title, s2_index)
df_season3 = to_df(s3_episode_no,s3_episode_title, s3_index)
df_season4 = to_df(s4_episode_no,s4_episode_title, s4_index)
df_season5 = to_df(s5_episode_no,s5_episode_title, s5_index)
df_season6 = to_df(s6_episode_no,s6_episode_title, s6_index)
df_season7 = to_df(s7_episode_no,s7_episode_title, s7_index)
df_season_inter = to_df(s_inter_episode_no,s_inter_episode_title, s_inter_index)

### 2.5.4. Bölüm verilerini birleştirme

In [63]:
df_total_episodes = pd.concat([df_season1,df_season2,df_season3,df_season4,
                     df_season5,df_season6,df_season7, df_season_inter], axis=0)

In [64]:
df_total_episodes.head()

,E,title
371,1,KICKOFF!!
372,2,An EMPTY road in India
373,3,GANGA AARTI in India
374,4,Battling Cows in India
375,5,CHAIN MAINTENANCE of a Royal Enfield Himalayan


In [75]:
# indexlere göre sıralama
df_total_episodes = df_total_episodes.sort_index()

### 2.5.5. Tekrarlayan bölümler

Bu bölümler listede iki defa geçiyor, ilgili sezonda ve ayrıca başka listelerde. Tekrarlara gerek olmayacak.

In [65]:
bike_index = data[data.season=='Bike reviews, maintenance & logistics'].index
pop_index = data[data.season=='Popular uploads'].index

In [71]:
data = data.drop(bike_index)

In [73]:
data= data.drop(pop_index)

## 2.6. Veri setinin son hali

Bölüm başlıkları ve sayılarından oluşan dataframe ile 

In [76]:
itchy_boots = pd.concat([data, df_total_episodes], axis=1)

In [77]:
itchy_boots.head()

,season,episode,date,time,rating,season_no,season_title,uploading_date,rate,E,title
0,SEASON 7 - Return to Africa,ITCHY BOOTS New Season: this is where I am go...,4 months,20,"s 1,237,258 views",7,Return to Africa,0,1237258,-1,ITCHY BOOTS New Season: this is where I am go...
1,SEASON 7 - Return to Africa,I need their help. |S7 - E2|,4 months,19,"s 1,548,717 views",7,Return to Africa,0,1548717,2,I need their help.
2,SEASON 7 - Return to Africa,No motorcyclists have gone here before |S7 - E3|,4 months,25,"s 905,116 views",7,Return to Africa,0,905116,3,No motorcyclists have gone here before
3,SEASON 7 - Return to Africa,How I ended up in a Berber village in Morocco ...,4 months,18,"s 790,900 views",7,Return to Africa,0,790900,4,How I ended up in a Berber village in Morocco
4,SEASON 7 - Return to Africa,Motorcycling solo into the Sahara Desert of Mo...,4 months,18,"s 648,198 views",7,Return to Africa,0,648198,5,Motorcycling solo into the Sahara Desert of Mo...


# 3. Veriyi kaydetme

In [78]:
itchy_boots.to_csv('datasets/itchy_boots.csv')